In [1]:
import numpy as np
import pandas as pd
from scipy.signal import stft
from scipy import stats
from collections import Counter
import matplotlib.pyplot as plt

In [2]:
def loadData(data_set = None):

    if (data_set == None):
        return None
    else:
        data = []
        fptr = open(data_set, "r")
        lines = fptr.readlines()
        for line in lines:
            line = line.strip().split()
            for i in range(len(line)):
                line[i] = float(line[i])
            # print(line)
            data.append(line)
        
        return np.array(data)

In [35]:
def giveFreqCoeff(data_set):
    
    x = data_set[:, 0]
    y = data_set[:, 1]
    z = data_set[:, 2]

    fs = 32
    seg_size = 32
    hop_size = seg_size // 2

    f_x, t_x, Zxx_x = stft(x, fs, nperseg=seg_size, noverlap=hop_size)
    f_y, t_y, Zxx_y = stft(y, fs, nperseg=seg_size, noverlap=hop_size)
    f_z, t_z, Zxx_z = stft(z, fs, nperseg=seg_size, noverlap=hop_size)

    # Magnitude of frequency coefficients for X, Y, Z-axis
    freq_coeffs_x = np.abs(Zxx_x)  
    freq_coeffs_y = np.abs(Zxx_y)  
    freq_coeffs_z = np.abs(Zxx_z)  

    freq_coeffs_x = freq_coeffs_x[0]
    freq_coeffs_y = freq_coeffs_y[0]
    freq_coeffs_z = freq_coeffs_z[0]

    return freq_coeffs_x, freq_coeffs_y, freq_coeffs_z, t_x[1] - t_x[0]


In [4]:
def mergeDataset(data_set, fc_x, fc_y, fc_z, time_diff, rating):

    new_data_set = []
    length = int(time_diff * 32)
    # = min(length, len(fc_x))

    time = 0
    idx = 0
    l = 0

    for i in range(0, len(data_set), length):

        temp = []
        x = data_set[i: i + 32, 0]
        y = data_set[i: i + 32, 1]
        z = data_set[i: i + 32, 2]
        op = data_set[i: i + 32, 3]
        
        
        find_most_common = lambda arr: max(Counter(arr).items(), key=lambda x: x[1])[0]
        op = find_most_common(op)

        # adding mean to data set
        temp.append(np.mean(x))
        temp.append(np.mean(y))
        temp.append(np.mean(z))

        # adding median to the data set
        temp.append(np.median(x))
        temp.append(np.median(y))
        temp.append(np.median(z))

        # adding mode to the data set
        temp.append(stats.mode(x, keepdims=True)[0][0])
        temp.append(stats.mode(y, keepdims=True)[0][0])
        temp.append(stats.mode(z, keepdims=True)[0][0])

        # adding the standard deviation to the data set
        temp.append(np.std(x))
        temp.append(np.std(y))
        temp.append(np.std(z))

        # adding the quantile @25 in the data set
        iq_1 = np.quantile(x, .25)
        iq_2 = np.quantile(y, .25)
        iq_3 = np.quantile(z, .25)
        temp.append(iq_1)
        temp.append(iq_2)
        temp.append(iq_3)

        # adding the quantile @50 in the data set
        temp.append(np.quantile(x, .5))
        temp.append(np.quantile(y, .5))
        temp.append(np.quantile(z, .5))

        # adding the quantile @75 in the data set
        iq_1_ = np.quantile(x, .75)
        iq_2_ = np.quantile(y, .75)
        iq_3_ = np.quantile(z, .75)
        temp.append(iq_1_)
        temp.append(iq_2_)
        temp.append(iq_3_)

        # adding inter quantile in the data set
        temp.append(iq_1_ - iq_1)
        temp.append(iq_2_ - iq_2)
        temp.append(iq_3_ - iq_3)


        temp.append(fc_x[l])
        temp.append(fc_y[l])
        temp.append(fc_z[l])

        # adding the output 
        # temp.append(op)

        temp.append(rating[idx])
        idx += 1

        new_data_set.append(temp)

        l += 1

        if idx == len(rating):
            break
        
        if l >= len(fc_x) or (i + 32) >= len(data_set):
            break

    
    return new_data_set

In [36]:
data_set_1 = loadData("Australia/visual_annotation_aus.txt")
rating_1 = pd.read_csv("Australia/Rating.csv")
rating_1 = rating_1["Rating"].to_numpy()

In [37]:
freq_coeffs_x, freq_coeffs_y, freq_coeffs_z, time_diff = giveFreqCoeff(data_set_1)

In [38]:
new_data_set = mergeDataset(data_set_1, freq_coeffs_x, freq_coeffs_y, freq_coeffs_z, time_diff, rating_1)

In [39]:
def witeNewData(new_data_set):

    # print("5:12")  
    file = open("visual_freq_rat.txt", "w") 
    for i in range(0, len(new_data_set)):
        for elem in new_data_set[i]:
            file.write(str(elem) + "\t")
        file.write("\n")
        # file.writelines(f"{new_data_set[i]}\t{new_data_set[i]}\t{new_data_set[i]}\t{new_data_set}\t\n")

    file.close()

witeNewData(new_data_set)

In [30]:
len(new_data_set)

17120

In [26]:
x = data_set_1[:, 0]
y = data_set_1[:, 1]
z = data_set_1[:, 2]

fs = 32
seg_size = 32
hop_size = seg_size // 2

f_x, t_x, Zxx_x = stft(x, fs, nperseg=seg_size, noverlap=hop_size)
f_y, t_y, Zxx_y = stft(y, fs, nperseg=seg_size, noverlap=hop_size)
f_z, t_z, Zxx_z = stft(z, fs, nperseg=seg_size, noverlap=hop_size)

In [ ]:
fig = plt.figure(figsize=(9, 5))
spec = plt.pcolormesh(t_x, f_x, np.abs(Zxx_x))
cbar = plt.colorbar(spec)

plt.title('STFT Spectrogram')
ax = fig.axes[0]
ax.grid(True)
ax.set_title('STFT Magnitude')

ax.set_ylabel('Frequency [Hz]')
ax.set_xlabel('Time [sec]')
fig.show

In [129]:

freq_coeffs_x = np.abs(Zxx_x)
freq_coeffs_x.shape
t_x[:5]

array([0. , 0.5, 1. , 1.5, 2. ])

In [12]:
# Magnitude of frequency coefficients for X, Y, Z-axis
freq_coeffs_x = np.abs(Zxx_x)  
freq_coeffs_y = np.abs(Zxx_y)  
freq_coeffs_z = np.abs(Zxx_z)  

(1397,)

In [172]:
freq_coeffs_z = np.mean(freq_coeffs_z, axis=0)


In [143]:
freq_coeffs_z[:10]

array([2.01334774, 2.6165116 , 2.08104168, 1.54167041, 1.50192039,
       2.74821563, 5.52708719, 3.43208069, 3.34656531, 4.25126541])

In [42]:
print(x, freq_coeffs_x)

[-0.25961899  0.38714278 -0.59462572 ...  0.02012856  2.98186382
 -0.37143739] [[0.12698784 0.0495531  0.10002736 ... 0.02178528 0.14104729 0.00299342]
 [0.11567441 0.04166631 0.09738974 ... 0.16406773 0.22797895 0.05506835]
 [0.08274473 0.0964906  0.11445873 ... 0.14021225 0.14410543 0.07773322]
 ...
 [0.03408262 0.1867545  0.0890008  ... 0.11873832 0.0926552  0.04138756]
 [0.0297205  0.22842615 0.0494959  ... 0.18131414 0.13060486 0.07925463]
 [0.01005903 0.26131465 0.01842983 ... 0.19528973 0.16200093 0.12032045]]


In [45]:
len(x)

3200

In [50]:
len(freq_coeffs_x[50])

65

In [13]:
freq_coeffs_x.shape

(51, 448)